### Clean Data

In [1]:
import pandas as pd
import numpy as np

Load `csv`s into Pandas DataFrame

In [2]:
salary = pd.read_csv('../data/cleaned/NBA_Salary.csv')
potw = pd.read_csv('../data/raw/NBA_Player_of_the_Week.csv')
stats = pd.read_csv('../data/scraped/NBA_Player_Stats.csv')
summary = pd.read_csv('../data/scraped/NBA_Yearly_Summary.csv')

In [3]:
print(f'Salary: {salary.shape[0]} observations, {salary.shape[1]} variables')
print(f'POTW: {potw.shape[0]} observations, {potw.shape[1]} variables')
print(f'Stats: {stats.shape[0]} observations, {stats.shape[1]} variables')
print(f'Summary: {summary.shape[0]} observations, {summary.shape[1]} variables')

Salary: 12415 observations, 6 variables
POTW: 1187 observations, 14 variables
Stats: 18480 observations, 30 variables
Summary: 35 observations, 8 variables


Rename Player Statistics columns

In [4]:
stats.columns = stats.columns.str.replace('%', '_Prct')
stats.columns = stats.columns.str.replace('2', 'Two_')
stats.columns = stats.columns.str.replace('3', 'Three_')

Rename the columns, remove duplicates, and assign `Potw` flag to all rows to Player of the Week data

In [5]:
potw = (potw.rename(columns = {'Season short': 'Year'})
        .groupby(['Player', 'Year'])[['Date']].count()
        .reset_index()[['Player', 'Year']]
        .assign(Potw = 1))

Merge the Player of the Week data to the Player Statistics data

In [6]:
nba_player = pd.merge(left = stats, right = potw, on = ['Player', 'Year'], how = 'left')
nba_player.Potw = nba_player.Potw.where((pd.notnull(nba_player.Potw)), 0)

Rename and stack the columns in Yearly Summary and make dummy variables

In [7]:
summary = summary.rename(columns = {'Most Valuable Player': 'MVP', 'Rookie of the Year': 'Rookie'})
summary = summary.set_index('Year').stack().reset_index().rename(columns = {'level_1': 'Type' , 0: 'Player'})
summary = summary.loc[~(summary.Type == 'League Champion')]
summary.Type = summary.Type.str.replace(' ', '_')
summary = (pd.concat([summary[['Year', 'Player']], pd.get_dummies(summary.Type)], axis = 1)
           .groupby(['Year', 'Player']).sum().reset_index())

Merge the Yearly Summary data to the rest of the data

In [8]:
nba_player = pd.merge(nba_player, summary, on = ['Player', 'Year'], how = 'left')
nba_player = nba_player.where((pd.notnull(nba_player)), 0)

Rename columns in Player Salary

In [9]:
salary = salary.rename(columns = {'Player Name': 'Player', 'Season End': 'Year', 'Salary in $': 'Salary', 'Team': 'Tm'})[['Player', 'Year', 'Salary', 'Tm']]

Merge the Player Salary data to the rest of the data

In [10]:
nba_player = pd.merge(nba_player, salary, on = ['Player', 'Year', 'Tm'], how = 'inner')
nba_player

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PF,PTS,Potw,APG_Leader,MVP,PPG_Leader,RPG_Leader,Rookie,WS_Leader,Salary
0,1991,Alaa Abdelnaby,PF,22.0,POR,43,0,6.7,1.3,2.7,...,0.9,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395000
1,1991,Mahmoud Abdul-Rauf,PG,21.0,DEN,67,19,22.5,6.2,15.1,...,2.2,14.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1660000
2,1991,Mark Acres,C,28.0,ORL,68,0,19.3,1.6,3.1,...,3.2,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,437000
3,1991,Michael Adams,PG,28.0,DEN,66,66,35.5,8.5,21.5,...,2.5,26.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,825000
4,1991,Mark Aguirre,SF,31.0,DET,78,13,25.7,5.4,11.7,...,2.7,14.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1115000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8998,2019,Delon Wright,PG,26.0,TOR,49,2,18.3,2.6,6.0,...,1.1,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2536898
8999,2019,Guerschon Yabusele,PF,23.0,BOS,41,1,6.1,0.9,1.9,...,0.8,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2667600
9000,2019,Thaddeus Young,PF,30.0,IND,81,81,30.7,5.5,10.4,...,2.4,12.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13764045
9001,2019,Trae Young,PG,20.0,ATL,81,81,30.9,6.5,15.5,...,1.7,19.1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5356440


In [11]:
nba_player.to_csv('../data/NBA_Players.csv', index = False)